# Portfolio Analysis

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime 
import yfinance as yf
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
# Set up dates
purchase_date = datetime(2023, 8, 21)
sell_date = datetime(2023, 11, 20)
month_one_end_date = datetime(2023, 9, 21)
month_two_end_date = datetime(2023, 10, 21)
month_three_end_date = datetime(2023, 11, 18)

In [ ]:
# Define methods
def get_stock_data(df_portfolio: pd.DataFrame) -> pd.DataFrame:
    """Get stock data from Yahoo Finance API for a list of tickers and a date range
    
    Args:
        df_portfolio: (pd.DataFrame): List of stock tickers
        
    Returns:
        pd.DataFrame: Stock data for tickers and date range
    """
    tickers = df_portfolio.index.to_list()
    stock_data: pd.DataFrame = yf.download(tickers, start=purchase_date, end=sell_date, interval="1d", auto_adjust=True, prepost=False, threads=True, proxy=None, group_by='ticker')
    
    # Set up date index for df_stock_data from Purchase_date to sell_date
    stock_data = stock_data.reindex(pd.date_range(start=purchase_date, end=sell_date, freq='D'))
    stock_data = stock_data.ffill()
    stock_data = stock_data.dropna()
    
    if len(tickers) == 1:       
        cols = pd.MultiIndex.from_tuples([
            (tickers[0],   'Open'),
            (tickers[0],   'High'),
            (tickers[0],    'Low'),
            (tickers[0],  'Close'),
            (tickers[0], 'Volume')
            ])
        stock_data.columns = cols
            
    for ticker in tickers:        
        # add new column for initial investment to df_stock_data for each group
        stock_data[ticker, 'Initial Investment'] = df_portfolio.loc[ticker, 'AMOUNT_INVESTED']
        stock_data[ticker, 'Purchase Price'] = stock_data.loc[purchase_date, (ticker, 'Close')]
        stock_data[ticker, 'Shares'] = stock_data[ticker, 'Initial Investment'] / stock_data[ticker, 'Purchase Price']
        stock_data[ticker, 'Current Value'] = stock_data[ticker, 'Shares'] * stock_data[ticker, 'Close']        
        stock_data[ticker, 'Percent Change'] = (stock_data[ticker, 'Close'] - stock_data[ticker, 'Purchase Price']) / stock_data[ticker, 'Purchase Price']
    
        
    return stock_data

def extend_portfolio_data(df_portfolio: pd.DataFrame, df_stock_data: pd.DataFrame) -> None:
    """Extend portfolio data with stock data
    
    Args:
        df_portfolio (pd.DataFrame): Portfolio data
        df_stock_data (pd.DataFrame): Stock data
        
    Returns:
        None
    """
    if 'EXPECTED_RETURN_ARR' in df_portfolio.columns:
        df_portfolio['ADJUSTED_EXPECTED_RETURN'] = df_portfolio['EXPECTED_RETURN_ARR'] / 4
    else:
        df_portfolio['EXPECTED_RETURN_ARR'] = np.NAN
        df_portfolio['ADJUSTED_EXPECTED_RETURN'] = np.NAN
    
    
    tickers = df_portfolio.index.to_list()
    for ticker in tickers:
        df_portfolio.loc[ticker, 'PURCHASE_PRICE'] = df_stock_data.loc[purchase_date, (ticker, 'Close')]
        df_portfolio.loc[ticker, 'SHARES'] = df_stock_data.loc[purchase_date, (ticker, 'Shares')]
        df_portfolio.loc[ticker, 'MONTH_1_PRICE'] = df_stock_data.loc[month_one_end_date, (ticker, 'Close')]
        df_portfolio.loc[ticker, 'MONTH_1_VALUE'] = df_stock_data.loc[month_one_end_date, (ticker, 'Current Value')]
        df_portfolio.loc[ticker, 'MONTH_1_RETURN'] = df_stock_data.loc[month_one_end_date, (ticker, 'Current Value')] - df_stock_data.loc[purchase_date, (ticker, 'Current Value')]
        df_portfolio.loc[ticker, 'MONTH_2_PRICE'] = df_stock_data.loc[month_two_end_date, (ticker, 'Close')]
        df_portfolio.loc[ticker, 'MONTH_2_VALUE'] = df_stock_data.loc[month_two_end_date, (ticker, 'Current Value')]
        df_portfolio.loc[ticker, 'MONTH_2_RETURN'] = df_stock_data.loc[month_two_end_date, (ticker, 'Current Value')] - df_stock_data.loc[month_one_end_date, (ticker, 'Current Value')]
        df_portfolio.loc[ticker, 'MONTH_3_PRICE'] = df_stock_data.loc[month_three_end_date, (ticker, 'Close')]
        df_portfolio.loc[ticker, 'MONTH_3_VALUE'] = df_stock_data.loc[month_three_end_date, (ticker, 'Current Value')]
        df_portfolio.loc[ticker, 'MONTH_3_RETURN'] = df_stock_data.loc[month_three_end_date, (ticker, 'Current Value')] - df_stock_data.loc[month_two_end_date, (ticker, 'Current Value')]
        df_portfolio.loc[ticker, 'ACTUAL_RETURN'] = df_stock_data.loc[month_three_end_date, (ticker, 'Current Value')] - df_stock_data.loc[purchase_date, (ticker, 'Current Value')]
        df_portfolio.loc[ticker, 'ACTUAL_RETURN_PCT'] = df_stock_data.loc[month_three_end_date, (ticker, 'Percent Change')]
        
# Function to calculate returns
def calculate_returns(df: pd.DataFrame, start: str, end: str) -> float:
    """Calculate returns for a given date range
    
    Args:
        df (pd.DataFrame): Stock data
        start (str): Start date
        end (str): End date
        
    Returns:
        float: Return for the date range        
    """
    subset = df.loc[start:end]
    start_value = subset['Current Value'].iloc[0]
    end_value = subset['Current Value'].iloc[-1]
    return (end_value - start_value) / start_value

def flatten_stock_data(df_stock_data: pd.DataFrame) -> pd.DataFrame:
    """Flatten stock data
    
    Args:
        df_stock_data (pd.DataFrame): Stock data
        
    Returns:
        pd.DataFrame: Flattened stock data
    """
    df_tmp = df_stock_data.stack(level=0).rename_axis(['Date', 'Ticker']).reset_index(level=1)
    df_flat_stock_data = df_tmp.groupby(level=0).sum()[['Current Value']]
    return df_flat_stock_data
    

In [ ]:
df_original_portfolio = pd.read_csv('../data/portfolio/original_portfolio.csv', index_col=0)
df_original_stock_prices = get_stock_data(df_original_portfolio)
extend_portfolio_data(df_original_portfolio, df_original_stock_prices)
df_flat_original_stock_prices = flatten_stock_data(df_original_stock_prices)

df_flat_original_stock_prices

In [ ]:
df_sentiment_portfolio = pd.read_csv('../data/portfolio/sentiment_portfolio.csv', index_col=0)
df_sentiment_stock_prices = get_stock_data(df_sentiment_portfolio)
extend_portfolio_data(df_sentiment_portfolio, df_sentiment_stock_prices)
df_flat_sentiment_stock_prices = flatten_stock_data(df_sentiment_stock_prices)

df_flat_sentiment_stock_prices

In [ ]:
df_djia_portfolio = pd.read_csv('../data/portfolio/djia_portfolio.csv', index_col=0)
df_djia_prices = get_stock_data(df_djia_portfolio)
df_flat_djia_prices = flatten_stock_data(df_djia_prices)

df_flat_djia_prices

In [ ]:
date_ranges = [('2023-08-22', '2023-09-21'), 
               ('2023-09-22', '2023-10-21'), 
               ('2023-10-21', '2023-11-18')]

# Calculate returns for each period
returns = []
for start_date, end_date in date_ranges:
    your_return = calculate_returns(df_flat_original_stock_prices, start_date, end_date)
    sentiment_return = calculate_returns(df_flat_sentiment_stock_prices, start_date, end_date)
    returns.append((start_date, end_date, your_return, sentiment_return))

# Create DataFrame for the returns
df_returns = pd.DataFrame(returns, columns=['Start Date', 'End Date', 'Your Portfolio Return', 'Sentiment Portfolio Return'])

# Display the DataFrame
df_returns

## Compare expected returns versus actual returns

In [ ]:
df_original_portfolio_returns = df_original_portfolio[['NAME', 'ADJUSTED_EXPECTED_RETURN', 'ACTUAL_RETURN_PCT']]

ind = np.arange(5)
width = 0.25
plt.figure(figsize=(10, 6))
bar1 = plt.bar(ind, df_original_portfolio_returns['ADJUSTED_EXPECTED_RETURN'], width, label='Expected Return')
bar2 = plt.bar(ind + width, df_original_portfolio_returns['ACTUAL_RETURN_PCT'], width, label='Actual Return')
plt.xlabel('Company')
plt.ylabel('% Return')
plt.title('Expected vs Actual Returns')

plt.xticks(ind + width / 2, df_original_portfolio_returns['NAME'])
plt.legend()
plt.show()

## Compare Original Portfolio vs Sentiment Portfolio

In [ ]:
df_original_portfolio["PORTFOLIO"] = "ORIGINAL"
df_sentiment_portfolio["PORTFOLIO"] = "SENTIMENT"
df_combined_portfolio = pd.concat([df_original_portfolio, df_sentiment_portfolio])
df_combined_portfolio

In [ ]:
df_combined_portfolio.groupby('PORTFOLIO').sum()[['MONTH_1_RETURN','MONTH_2_RETURN','MONTH_3_RETURN','ACTUAL_RETURN', 'ACTUAL_RETURN_PCT']]

In [ ]:
# Plotting
plt.figure(figsize=(10, 6))
plt.plot(df_returns['End Date'], df_returns['Your Portfolio Return'], label='Your Portfolio')
plt.plot(df_returns['End Date'], df_returns['Sentiment Portfolio Return'], label='Sentiment Portfolio')
plt.xlabel('End Date')
plt.ylabel('Return')
plt.title('Month-by-Month Portfolio Returns Comparison')
plt.legend()
plt.show()

In [ ]:
fig = go.Figure(
    [go.Scatter(x=df_flat_sentiment_stock_prices.index, y=df_flat_sentiment_stock_prices['Current Value'], name='Sentiment Portfolio'),
     go.Scatter(x=df_flat_original_stock_prices.index, y=df_flat_original_stock_prices['Current Value'], name='Team 6 Original Portfolio'),
     go.Scatter(x=df_flat_djia_prices.index, y=df_flat_djia_prices['Current Value'], name='DJIA')]
)
fig.update_layout(    
    xaxis_title='Date',
    yaxis_title='Portfolio Value'
)
fig.show()